In [2]:

# Preparing Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import*
# Load the rock mines dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
df = pd.read_csv(url)
column_names = ["sensor_" + str(i) for i in range(1, 61)] + ["target"]
df.columns = column_names
df = df.reset_index()
df = df.rename(columns={"index": "id"})
df['target'] = df['target'].map({'M': 1, 'R': 0})
x_data = df.iloc[:, :60]
x_data = x_data.drop('id', axis=1)
y_data = df['target']
x_data['encoded_column'] = (x_data['sensor_2'] < 0.02).astype(int)


def apply_transformer_and_suffix(dataframe, transformer, suffix, **kwargs):
    # Identify continuous columns (float or int data type)
    continuous_columns = dataframe.select_dtypes(include=['float64']).columns
    
    # Create an instance of the specified transformer with additional parameters
    scaler = transformer(**kwargs)
    scaled_data = scaler.fit_transform(dataframe[continuous_columns])
    
    # Create new column names with the specified suffix
    new_columns = [col + f"_{suffix}" for col in continuous_columns]
    
    # Create a new DataFrame with transformed data and modified column names
    transformed_df = pd.DataFrame(scaled_data, columns=new_columns)
    
    # Combine transformed continuous columns with non-continuous columns
    non_continuous_columns = dataframe.drop(columns=continuous_columns)
    final_df = pd.concat([non_continuous_columns, transformed_df], axis=1)
    
    return final_df



StandardScaler_df = apply_transformer_and_suffix(x_data, StandardScaler, "StandardScaler")
Normalizer_df = apply_transformer_and_suffix(x_data, Normalizer, "Normalizer")
MaxAbsScaler_df = apply_transformer_and_suffix(x_data, MaxAbsScaler, "MaxAbsScaler")
MinMaxScaler_df = apply_transformer_and_suffix(x_data, MinMaxScaler, "MinMaxScaler")
QuantileTransformer_df = apply_transformer_and_suffix(x_data, QuantileTransformer, "QuantileTransformer", n_quantiles=10)
KBinsDiscretizer_df = apply_transformer_and_suffix(x_data, KBinsDiscretizer, "KBinsDiscretizer",n_bins=10, encode='ordinal', strategy='uniform')
PowerTransformer_df = apply_transformer_and_suffix(x_data, PowerTransformer, "PowerTransformer",method='yeo-johnson', standardize=True)
RobustScaler_df = apply_transformer_and_suffix(x_data, RobustScaler, "RobustScaler",with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0))


all_dfs = [StandardScaler_df, Normalizer_df, MaxAbsScaler_df, MinMaxScaler_df,QuantileTransformer_df,KBinsDiscretizer_df,PowerTransformer_df,RobustScaler_df]
# Initialize an empty DataFrame to store the merged result
merged_df = pd.DataFrame()
# Concatenate DataFrames vertically, avoiding duplicate columns
for df in all_dfs:
    new_columns = [col for col in df.columns if col not in merged_df.columns]
    merged_df = pd.concat([merged_df, df[new_columns]], axis=1)

